In [1]:
# 기본
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import gc

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 그래프 설정
sns.set()

# 그래프 기본 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
# plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['figure.figsize'] = 12, 6
plt.rcParams['font.size'] = 14
plt.rcParams['axes.unicode_minus'] = False

# 결측치 시각화를 위한 라이브러리
import missingno

from functools import reduce

In [2]:
# 1–8번 파일 읽어와서 train/test 합치기
paths = [
    ('회원정보_preprocessing.csv'),
    ('신용정보_preprocessing.csv'),
    ('승인매출정보_전처리.csv'),
    ('청구잔액전처리_최종데이터.csv'),
    ('채널정보_전처리.csv'),
    ('마케팅정보_전처리.csv'),
    ('성과정보_preprocessing.csv'),
]

In [3]:
dfs = []
for path in paths:
    # 추출: CSV 파일 읽기
    df = pd.read_csv(path)
    
    # 정의: '기준년월' 컬럼이 있으면 'YYYY-MM' 형식으로 변환
    if '기준년월' in df.columns:
        df['기준년월'] = pd.to_datetime(
            df['기준년월'].astype(str),
            format='%Y%m',
            errors='coerce'
        ).dt.strftime('%Y-%m')
    
    dfs.append(df)

# 12월 데이터만 필터링
dfs_dec = [df[df['기준년월'].str.endswith('-12')].copy() for df in dfs]

# 중복된 Segment 열 제거 (첫 번째 데이터프레임만 Segment 유지)
for df in dfs_dec[1:]:
    if 'Segment' in df.columns:
        df.drop('Segment', axis=1, inplace=True)

# 공통 키(ID, 기준년월)로 병합
merged_df = reduce(
    lambda left, right: pd.merge(left, right, on=['ID', '기준년월'], how='outer'),
    dfs_dec
)

# ID가 'TRAIN'으로 시작하는 행은 train 데이터로 분리
train_df = merged_df[merged_df['ID'].str.startswith('TRAIN')].copy()

# ID가 'TEST'로 시작하는 행은 test 데이터로 분리
test_df  = merged_df[merged_df['ID'].str.startswith('TEST')].copy()

# 확인: 분리된 데이터프레임 크기 출력
print(f"Train 데이터 수: {len(train_df)}")
print(f"Test 데이터 수:  {len(test_df)}")

Train 데이터 수: 400000
Test 데이터 수:  100000


In [4]:
# train_df와 test_df의 컬럼별 데이터 타입별 개수 확인
for name, df_part in [('train_df', train_df), ('test_df', test_df)]:
    print(f"\n{name} 컬럼 데이터 타입별 개수:")
    print(df_part.dtypes.value_counts())


train_df 컬럼 데이터 타입별 개수:
int64      626
bool       104
float64     57
object       3
Name: count, dtype: int64

test_df 컬럼 데이터 타입별 개수:
int64      626
bool       104
float64     57
object       3
Name: count, dtype: int64


In [5]:
# train_df의 object 타입 컬럼 리스트
obj_cols_train = train_df.select_dtypes(include='object').columns.tolist()

# train_df object 컬럼 및 개수
print(f"train_df object 컬럼 ({len(obj_cols_train)}개):")
print(obj_cols_train)

# test_df의 object 타입 컬럼 리스트
obj_cols_test = test_df.select_dtypes(include='object').columns.tolist()

# test_df object 컬럼 및 개수
print(f"\ntest_df object 컬럼 ({len(obj_cols_test)}개):")
print(obj_cols_test)

train_df object 컬럼 (3개):
['기준년월', 'ID', 'Segment']

test_df object 컬럼 (3개):
['기준년월', 'ID', 'Segment']


In [9]:
# 저장: 결과를 CSV로 내보내기
train_df.to_csv('merged_2018_train.csv', index=False)
test_df.to_csv('merged_2018_test.csv',  index=False)